Exploratory Data Analysis:  MLS Salary 2007 - 2017
--------------------------------------------------

This is my first submission to Kaggle where I analyze Major League Soccer player salaries and how they have changed since 2007.  Any comments and feedback are welcome. 

 1. Import the Data
 2. Clean the Data
 3. Analyze the Data
 4. Plot the Data (Still In-Progress)

## Importing the Data ##

I imported all 11 csv files of data and concatenated them into one data frame, named **df_mls**.

In [ ]:
# Import the necessary Python libraries
import numpy as np
import pandas as pd 
import glob, os.path
import matplotlib.pyplot as plt
%matplotlib inline
from subprocess import check_output

In [ ]:
# Import all 11 csv files and concatenate them into one dataframe
path = r'../input/'
all_csv_files = glob.glob(os.path.join(path, "*.csv"))
dfs = [pd.read_csv(i).assign(Season=os.path.basename(i)) for i in all_csv_files]
df_mls = pd.concat(dfs, ignore_index=True)

In [ ]:
# A new column named "Season" will be created and added to the new dataframe
df_mls['Season'] = df_mls['Season'].str.replace('mls-salaries-', '')
df_mls['Season'] = df_mls['Season'].str.replace('.csv', '')

In [ ]:
# Observe the dataframe
df_mls.head()

In [ ]:
# Check for any missing values
df_mls.count()

In [ ]:
# Observe the missing data
null_data = df_mls[df_mls.isnull().any(axis=1)]
null_data

**After looking at the null values, I came to the following conclusions:**

 - The null values for **'first_name'** can be explained by certain players going by only one name, such as Kaka. 
 - The null values for the column **'club'** can be explained by them being  a free agent. 
 - The null values for **'position'** and **'guaranteed_salary'** and **'base_salary'** are due to missing data.

**To Resolve the Null Values for 'first_name':**

I wrote a function to combine the **'first_name'** and **'last_name'** columns together which also takes into account when players have only a first name and add this value to a new column **'Name'**.

In [ ]:
# Create a new Column "Name" that combines first and last name together 
def fullname(x, y):
    if str(x) == "nan":
        return str(y)
    else:
        return str(x) + " " + str(y)

In [ ]:
# Apply fullname functon to the dataframe
df_mls['Name'] = np.vectorize(fullname)(df_mls['first_name'], df_mls['last_name'])

In [ ]:
# Check to see if the function worked sucessfully
df_mls.head()

In [ ]:
# Drop the 'last_name' and 'first_name' columns
df_mls = df_mls.drop(['last_name', 'first_name'], axis = 1)

In [ ]:
# Rearrange the order of the columns
df_mls = df_mls[['Season', 'club', 'Name', 'position', 'guaranteed_compensation', 'base_salary']]
df_mls.head()

 **To Resolve the Null Values and Odd Values for 'club':**

I wrote function that assigns free agent players, which I defined as players with 'club' being 'Nan', 'None',  'POOL' or 'Pool'. I researched what 'POOL' and 'Pool' referred to and it refers to players who are contracted to MLS, but do not play for a club on a game to game basis. I also created a new data frame for these players, named **df_free_agents,** and removed them from the main data frame, **df_mls**.

In [ ]:
# Observe the different 'club' names
df_mls['club'].unique()

In [ ]:
# Investigating what 'pool' is
df_mls[df_mls['club'] == 'Pool']

In [ ]:
df_mls[df_mls['club'] == 'POOL']

In [ ]:
# Create a function to assign rows to "Free Agent"
def free_agent(x):
    if str(x) == "nan":
        return str("Free Agent")
    elif str(x) == 'None':
        return str("Free Agent")
    elif str(x) == 'Pool':
        return str("Free Agent")
    elif str(x) == 'POOL':
        return str("Free Agent")
    else:
        return str(x)

In [ ]:
# Apply the free_agent function to the dataframe
df_mls['club'] = df_mls['club'].apply(free_agent)

In [ ]:
# Create a new dataframe for free agents and observe the new dataframe
df_free_agents = df_mls[df_mls['club'] == 'Free Agent']
df_free_agents.head(10)

In [ ]:
# Remove 'Free Agents' from the dataframe  
df_mls.drop(df_mls[df_mls['club'] == 'Free Agent'].index, inplace=True)

In [ ]:
# Drop rows with missing base_salary
df_mls = df_mls[pd.notnull(df_mls['base_salary'])]

**To Resolve the Null Value for Position**

There was one row with a missing value for 'position'. Since it is one player, I researched and found out he is a defender and a midfielder; D/M

In [ ]:
# Observe the row with missing position
df_mls[df_mls['position'].isnull()] 


In [ ]:
# Assign the missing value the correct value
df_mls.loc[873, 'position'] = str('D/M')

**Data Wranging for the Position Values:**

I wanted the  'position' values to be more consistent so I wrote a function that converts different position combinations into a consistent combination value. For example, instead of having 'MF', 'M-F', 'F/M', or  'M/F' as a position value, the function converts these to 'M/F'.   

In [ ]:
# Observe the different positions
df_mls['position'].unique()

In [ ]:
# Create a function to correct positions
def position_fixer(x):
    if x == 'D-M':
        return str('D/M')
    if x == 'F-D':
        return str('D/F')
    if x == 'D-F':
        return str('D/F')
    if x == 'F-M':
        return str('M/F')
    if x == 'M-F':
        return str('M/F')
    if x == 'M-D':
        return str('D/M')
    if x == 'M/D':
        return str('D/M')
    if x == 'MF':
        return str('M/F')
    if x == 'F/M':
        return str('M/F')
    else:
        return str(x)

In [ ]:
# Apply the position_fixer function to the dataframe
df_mls['position'] = df_mls['position'].apply(position_fixer)

 **Data Cleaning/Wrangling Conclusion:**

The new dataframe **df_mls**  no longer has any nulls and is consistent with its values. It is ready for some exploratory analysis.

In [ ]:
# Observe the new dataframe
df_mls.count()

In [ ]:
df_mls.describe()

In [ ]:
df_mls.head(10)

## Exploratory Analysis ##

I am interested in observing how salary has changed over the 11 seasons.  I analyzed how salaries have changed over the 11 seasons. I looked trends by season, by club,  by position, and by the league overall.

In [ ]:
# Observe the Average Salary by Season
df_mls.groupby(by = 'Season')['guaranteed_compensation'].mean()

In [ ]:
# Observe the Max Salary by Season
df_mls.groupby(by = 'Season')['guaranteed_compensation'].max()

In [ ]:
# Observe the Min Salary by Season
df_mls.groupby(by = 'Season')['guaranteed_compensation'].min()

In [ ]:
# Observe the Difference between the Max and Min Salary by Season
df_mls.groupby(by = 'Season')['guaranteed_compensation'].max() - df_mls.groupby(by = 'Season')['guaranteed_compensation'].min()

In [ ]:
# There are some potential outliers with this data
df_mls.groupby(by = 'Season')['guaranteed_compensation'].median()

In [ ]:
# Highest Paid Player per season
df_mls.sort_values('guaranteed_compensation', ascending=False).groupby('Season', as_index=False).first()

In [ ]:
# Highest Paid Player per Position 2007-2017
df_mls.sort_values('guaranteed_compensation', ascending=False).groupby('position', as_index=False).first()

The highest paid players of the current 2017 season

In [ ]:
df_top17 = df_mls[df_mls['Season'] == '2017']
df_top17.sort_values('guaranteed_compensation', ascending = False).head(15)

In [ ]:
# Ten Highest Paid Players 2007 - 2017
df_final = df_mls.drop_duplicates(subset='Name')
df_final.sort_values('guaranteed_compensation', ascending = False).head(10)

In [ ]:
# Highest Paid Player Per Club Each Season
df_mls.sort_values('guaranteed_compensation', ascending=False).groupby(['Season', 'club'], as_index=False).first()

In [ ]:
# Highest Paid Player Per Club 2007 to 2017 Season
df_mls.sort_values('guaranteed_compensation', ascending=False).groupby('club', as_index=False).first()

In [ ]:
df_free_agents['guaranteed_compensation'].mean()

In [ ]:
# Highest Paid Free Agent Per Season
df_free_agents.sort_values('guaranteed_compensation', ascending=False).groupby('Season', as_index=False).first()

In [ ]:
# Highest Paid Free Agent Per Season
df_free_agents.sort_values('guaranteed_compensation', ascending=False).groupby('Season', as_index=False).first()

## Plotting the Data ##

In [ ]:
salary_totals = df_mls.groupby('Season').sum()
salary_totals.plot(kind = 'bar', title = "Total Salary by Season",figsize=(10,6)).set_ylabel("Salary (USD in millions)");
# The total salary has increased over the years


The total salary paid by each club has generally increased over the eleven seasons. There was a decrease from Season 2012 to 2013, which was also the season David Beckham no longer was playing in the league. However, from Season 2013 onward, the total salary continued to increase. 

In [ ]:
salary_median = df_mls.groupby('Season').median()
salary_median.plot(kind = 'bar', title = "Median Salary by Season",figsize=(10,6)).set_ylabel("Salary (USD in millions)");

In [ ]:
salary_mean = df_mls.groupby('Season').mean()
salary_mean.plot(kind = 'bar', title = "Average Salary by Season",figsize=(10,6)).set_ylabel("Salary (USD in millions)");

In [ ]:
salary_totals_by_club = df_mls.groupby('club').sum()
salary_totals_by_club.plot(kind = 'bar', title = "Total Salary by Club",figsize=(10,6)).set_ylabel("Salary (USD in millions)");

In [ ]:
df_mls17 = df_mls[df_mls['Season'] == '2017'].sort_values('guaranteed_compensation').groupby('club').sum()
df_mls17.plot(kind = 'bar', title = "2017 Total Salary by Club",figsize=(10,6)).set_ylabel("Salary (USD in millions)");

I wanted to observe how much each club has spent on player salaries over the 11 seasons. LA Galaxy, New York Red bulls, and Toronto FC have been the biggest spenders. Interestingly, New York FC, which was had its first season in MLS in 2013 has already spent more than five other clubs. 

Please note: LAFC is a new expansion club and will enter the league in 2018.  

In [ ]:
df_mls_dc = df_mls[df_mls['club'] == "DC"]
df_max_salary_by_season_dc = df_mls_dc.groupby('Season')['guaranteed_compensation'].max()
df_max_salary_by_season_dc.plot(kind = 'bar', title = "Highest Salary by Season (DC United)",figsize=(10,6), color = 'red').set_ylabel("Salary (USD in millions)");

I wanted to see how my local club, DC United, have been spending.

In [ ]:
df_mls_la = df_mls[df_mls['club'] == "LA"]
df_max_salary_by_season_la = df_mls_la.groupby('Season')['guaranteed_compensation'].max()
df_max_salary_by_season_la.plot(kind = 'bar', title = "Highest Salary by Season (La Galaxy)",figsize=(10,6), color = 'purple').set_ylabel("Salary (USD in millions)");